In [116]:
import synthetic_data as sd
import network_builder as nb
import networkx as nx
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

In [45]:
data = sd.synthetic_data_generator()

In [80]:
data.shape

(1000, 100)

In [190]:
n = data.shape[1]
m = nb.weighted_matrix(data[:999,:])
G, c = nb.graph_builder_limit(m, 0.3)
G = G.to_undirected()
G.remove_edges_from(nx.selfloop_edges(G))

degree = dict(G.degree())
closseness = nx.closeness_centrality(G)
kcore = dict(nx.core_number(G))
betweeness= dict(nx.betweenness_centrality(G))
pagerank = dict(nx.pagerank(G, alpha=0.85))
eigenvector = dict(nx.eigenvector_centrality(G, max_iter = 1000))


In [191]:
X = []
for i in range(n):
    if (i in degree.keys()):
        x = []
        #x.append(i)
        x.append(degree[i])
        x.append(closseness[i])
        x.append(kcore[i])
        x.append(betweeness[i])
        x.append(pagerank[i])
        x.append(eigenvector[i])
        x.append(nx.clustering(G, i))
        X.append(x)

In [192]:
Y = data[999,:]

In [193]:
clf = MLPRegressor(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(50, 25, 10, ), random_state=1)
clf.fit(X, Y)

MLPRegressor(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 25, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [194]:
y = clf.predict(X)

In [195]:
r2_score(Y,y) #0.75

0.06490654604107782

In [72]:
r2_score(Y,y) #0.9

-5.722089468918057e-13

In [65]:
r2_score(Y,y) #0.7

0.09408914142801561

In [59]:
r2_score(Y,y) #0.3

2.220446049250313e-16

In [114]:
r2_score(Y,y) #0.5

0.06490654604107782

In [196]:
regr = RandomForestRegressor(max_depth=2, random_state=0,n_estimators=100)
regr.fit(X, Y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [197]:
y = regr.predict(X)

In [198]:
r2_score(Y,y) #0.75

0.28165933123468556

In [ ]:
import numpy.ma as ma
import math
import numpy as np
from netCDF4 import Dataset
from scipy import stats
import networkx as nx
import pandas as pd


f_temp = Dataset('temp0818.nc')
f_pre = Dataset('pre0818.nc')

air = f_temp.variables['air']
pr_wtr = f_pre.variables['pr_wtr']

N = f_temp['time'].shape[0]

air = np.flip(air, 1)
pr_wtr = np.flip(pr_wtr, 1)

f_spei = Dataset('spei12.nc')
spei_data = f_spei.variables['spei']

def weighted_matrix(data, spei_data):
    lat_number = data.shape[1]
    lon_number = data.shape[2]
    N = lat_number * lon_number
    pearson_r = np.zeros((N,N))
    
    for i in range(N):
        lat_index = math.floor(i/lon_number)
        lon_index = i%lon_number
        if ma.is_masked(data[-1,lat_index ,lon_index]) or np.isnan(spei_data[-1,lat_index,lon_index]): 
            pearson_r[i,:] = np.nan
            continue;
        for j in range(N):
            lat_index_sec = math.floor(j/lon_number)
            lon_index_sec = j%lon_number
            if ma.is_masked(data[-1,lat_index_sec ,lon_index_sec]) or np.isnan(spei_data[-1,lat_index_sec,lon_index_sec]) or data[-1,lat_index_sec ,lon_index_sec]==-9.96921e+36:
                pearson_r[i,j] = np.nan
            else:
                pearson_r[i,j] = stats.pearsonr(data[:,lat_index ,lon_index],data[:,lat_index_sec ,lon_index_sec])[0]
                
    new = int(math.sqrt((pearson_r.shape[0] * pearson_r.shape[1]) - np.count_nonzero(np.isnan(pearson_r))))
    pearson_r_clean = pearson_r[~np.isnan(pearson_r)].reshape((new,new))
    return(pearson_r, pearson_r_clean)

def graph_builder (weighted_matrix):
    weighted_matrix = np.exp(-np.sqrt(1 - weighted_matrix))
    componenets_number = 0
    limit = 1.0
    while componenets_number != 1:
        limit -= 0.01
        adjacency_matrix = np.zeros(weighted_matrix.shape)
        adjacency_matrix[weighted_matrix >= limit] = 1
        G = nx.from_numpy_matrix(adjacency_matrix)
        G = G.to_undirected()
        Gcc = sorted(nx.connected_component_subgraphs(G), key = len, reverse=True)
        componenets_number = len(Gcc)
    return(G)

def degree_distribution(G):
    vk = dict(G.degree())
    vk = list(vk.values()) # we get only the degree values
    maxk = np.max(vk)
    mink = np.min(min)
    kvalues= np.arange(0,maxk+1) # possible values of k
    Pk = np.zeros(maxk+1) # P(k)
    for k in vk:
        Pk[k] = Pk[k] + 1
    Pk = Pk/sum(Pk) # the sum of the elements of P(k) must to be equal to one
    return kvalues,Pk

def momment_of_degree_distribution(G,m):
    k,Pk = degree_distribution(G)
    M = sum((k**m)*Pk)
    return M

def shannon_entropy(G):
    k,Pk = degree_distribution(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*math.log(p, 2)
    return H


curr_year = 2008
spei_start = 1295

year = []
average_degree_pr = []
second_moment_pr = []
variance_pr = []
shannon_entropy_pr = []
transitivity_pr = []
average_cluster_pr = []
average_shortest_path_length_pr = []
diameter_pr = []
global_efficiency_pr = []
local_efficiency_pr = []
average_closeness_pr = []
average_betweennes_pr = []
average_eigenvector_pr = []
average_pagerank_pr = []
assortativity_pr = []

average_degree_air = []
second_moment_air = []
variance_air = []
shannon_entropy_air = []
transitivity_air = []
average_cluster_air = []
average_shortest_path_length_air = []
diameter_air = []
global_efficiency_air = []
local_efficiency_air = []
average_closeness_air = []
average_betweennes_air = []
average_eigenvector_air = []
average_pagerank_air = []
assortativity_air = []

lon = np.arange(-40.0,72.0,2.5)
lat = np.arange(-50.0,51.0,2.5)
X = []
lat_number = len(lat)
lon_number = len(lon)

for z in range(math.floor(N/365)):
    
    spei = np.zeros((lat_number,lon_number))
    for i in range(79,280,5):
        for j in range(279,500,5):
            lat_index = int((i-79)/5)
            lon_index = int((j-279)/5)
            spei_index = spei_start + z*12
            if ma.is_masked(spei_data[-1,i,j]) or ma.is_masked(spei_data[-1,i,j+1]) or ma.is_masked(spei_data[-1,i+1,j]) or ma.is_masked(spei_data[-1,i+1,j+1]):
                spei[lat_index,lon_index]= np.nan
            else:
                spei[lat_index,lon_index] = (np.mean(spei_data[spei_index:spei_index + 12,i,j]) + np.mean(spei_data[spei_index:spei_index + 12,i,j+1]) 
                                           + np.mean(spei_data[spei_index:spei_index + 12,i+1,j]) + np.mean(spei_data[spei_index:spei_index + 12,i+1,j+1]))/4
    
    pr_matrix, pr_weighted_matrix = weighted_matrix(pr_wtr[z*365:z*365 + 365,:,:], spei_data)
       
    G_pr = graph_builder(pr_weighted_matrix)
    G_pr = G_pr.to_undirected()
    G_pr.remove_edges_from(nx.selfloop_edges(G_pr))
    
    vk_pr = dict(G_pr.degree())
    CLC_pr = nx.closeness_centrality(G_pr)
    KC_pr = dict(nx.core_number(G_pr))
    B_pr = dict(nx.betweenness_centrality(G_pr))
    PR_pr = dict(nx.pagerank(G_pr, alpha=0.85))
    EC_pr = dict(nx.eigenvector_centrality(G_pr, max_iter = 1000))
                  
    year.append(curr_year)
    
    average_degree_pr.append(momment_of_degree_distribution(G_pr,1))                  #First Moment
    second_moment_pr.append(momment_of_degree_distribution(G_pr,2))                   #Second Moment
    variance_pr.append(momment_of_degree_distribution(G_pr,2) - momment_of_degree_distribution(G_pr,1)**2)     #Variance
    shannon_entropy_pr.append(shannon_entropy(G_pr))                                  #Shanon Entropy
    transitivity_pr.append(nx.transitivity(G_pr))                                     #Transitivity 
    average_cluster_pr.append(nx.average_clustering(G_pr))
    #if nx.is_connected(G_pr) == True:
    average_shortest_path_length_pr.append(nx.average_shortest_path_length(G_pr))     #Average Shortest Path
    diameter_pr.append(nx.diameter(G_pr))                                             #Diameter   
    global_efficiency_pr.append(nx.global_efficiency(G_pr))                           #Efficiency
    local_efficiency_pr.append(nx.local_efficiency(G_pr))                             #Average Local Efficiency  
    average_closeness_pr.append(np.mean(list(CLC_pr.values())))                       #Average closeness centrality
    average_betweennes_pr.append(np.mean(list(B_pr.values())))                        #Average betweenness centrality
    average_eigenvector_pr.append(np.mean(list(EC_pr.values())))                      #Average eigenvector centrality
    average_pagerank_pr.append(np.mean(list(PR_pr.values())))                         #Average PageRank Centrality
    assortativity_pr.append(nx.degree_assortativity_coefficient(G_pr))                #Assortativity
    
    
    air_matrix, air_weighted_matrix = weighted_matrix(air[z*365:z*365 + 365,:,:], spei_data)
    
    G_air = graph_builder(air_weighted_matrix)
    G_air = G_air.to_undirected()
    G_air.remove_edges_from(nx.selfloop_edges(G_air))
    
    vk_air = dict(G_air.degree())
    CLC_air = nx.closeness_centrality(G_air)
    KC_air = dict(nx.core_number(G_air))
    B_air = dict(nx.betweenness_centrality(G_air))
    PR_air = dict(nx.pagerank(G_air, alpha=0.85))
    EC_air = dict(nx.eigenvector_centrality(G_air, max_iter = 1000))
                  
    average_degree_air.append(momment_of_degree_distribution(G_air,1))                  #First Moment
    second_moment_air.append(momment_of_degree_distribution(G_air,2))                   #Second Moment
    variance_air.append(momment_of_degree_distribution(G_air,2) - momment_of_degree_distribution(G_air,1)**2)     #Variance
    shannon_entropy_air.append(shannon_entropy(G_air))                                  #Shanon Entropy
    transitivity_air.append(nx.transitivity(G_air))                                     #Transitivity 
    average_cluster_air.append(nx.average_clustering(G_air))
    #if nx.is_connected(G_air) == True:
    average_shortest_path_length_air.append(nx.average_shortest_path_length(G_air))     #Average Shortest Path
    diameter_air.append(nx.diameter(G_air))                                             #Diameter   
    global_efficiency_air.append(nx.global_efficiency(G_air))                           #Efficiency
    local_efficiency_air.append(nx.local_efficiency(G_air))                             #Average Local Efficiency fficiency  
    average_closeness_air.append(np.mean(list(CLC_air.values())))          #Average closeness centrality
    average_betweennes_air.append(np.mean(list(B_air.values())))           #Average betweenness centrality
    average_eigenvector_air.append(np.mean(list(EC_air.values())))         #Average eigenvector centrality
    average_pagerank_air.append(np.mean(list(PR_air.values())))
    assortativity_air.append(nx.degree_assortativity_coefficient(G_air))                    #Assortativity   
    
    rows,cols = np.where(~np.isnan(pr_matrix))
    nodes = np.unique(rows)
    for i in range(np.unique(rows).shape[0]):
        lat_index = math.floor(nodes[i]/lon_number)
        lon_index = nodes[i] % lon_number
        if (i in vk_pr.keys()) and (np.count_nonzero(np.isnan(spei[:,lat_index,lon_index])) == 0):
            x = []
            x.append(curr_year)
            x.append(lat_index)
            x.append(lon_index)
            x.append(vk_air[i])
            x.append(CLC_air[i])
            x.append(KC_air[i])
            x.append(B_air[i])
            x.append(PR_air[i])
            x.append(EC_air[i])
            x.append(nx.clustering(G_air, i))
            x.append(vk_pr[i])
            x.append(CLC_pr[i])
            x.append(KC_pr[i])
            x.append(B_pr[i])
            x.append(PR_pr[i])
            x.append(EC_pr[i])
            x.append(nx.clustering(G_pr, i))
            for k in range(13):
                x.append(spei[k,lat_index,lon_index])
            X.append(x)
                
    curr_year += 1

df_X = pd.DataFrame(X,columns=['year','lat','lon','vk_air','CLC_air','KC_air','B_air','PR_air','EC_air','clustering_air',
                                'vk_pr','CLC_pr','KC_pr','B_pr','PR_pr','EC_pr','clustering_pr','SPEI'])
    
df_pr = pd.DataFrame(list(zip(year, average_degree_pr, second_moment_pr, variance_pr, shannon_entropy_pr, transitivity_pr,
                           average_cluster_pr, average_closeness_pr, average_betweennes_pr, average_eigenvector_pr,
                           average_pagerank_pr, assortativity_pr,average_shortest_path_length_pr,diameter_pr,global_efficiency_pr,local_efficiency_pr)),
                 columns=['year', 'average_degree_pr', 'second_moment_pr', 'variance_pr', 'shannon_entropy_pr',
                          'transitivity_pr', 'average_cluster_pr', 'average_closeness_pr', 'average_betweennes_pr',
                          'average_eigenvector_pr', 'average_pagerank_pr','assortativity_pr','average_shortest_path_length_pr','diameter_pr,global_efficiency_pr','local_efficiency_pr'])
df_air = pd.DataFrame(list(zip(year, average_degree_air, second_moment_air, variance_air, shannon_entropy_air, transitivity_air,
                           average_cluster_air, average_closeness_air, average_betweennes_air, average_eigenvector_air,
                           average_pagerank_air, assortativity_air,average_shortest_path_length_air,diameter_air,global_efficiency_air,local_efficiency_air)),
                 columns=['year', 'average_degree_air', 'second_moment_air', 'variance_air', 'shannon_entropy_air',
                          'transitivity_air', 'average_cluster_air','average_closeness_air', 'average_betweennes_air',
                          'average_eigenvector_air', 'average_pagerank_air','assortativity_air','average_shortest_path_length_air','diameter_air','global_efficiency_air','local_efficiency_air'])
    
df_pr.to_csv('pr.csv')    
df_air.to_csv('air.csv')
df_X.to_csv('Xdata.csv')